In [1]:
import os
import json
import torch

from utils import find_assistant_token_end
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model = AutoModelForCausalLM.from_pretrained("/nlsasfs/home/isea/isea10/aansh/deception_detection/weights/Llama-3.1-8B-Instruct", device_map="auto", dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("/nlsasfs/home/isea/isea10/aansh/deception_detection/weights/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
with open("/nlsasfs/home/isea/isea10/aansh/introspection/data/exp1/multi_token_completions/simple_narrative_continuations_category_1.json", "r") as f:
    data = json.load(f)
    
example = data[0]

In [4]:
user_prompt = f"""
Complete the following sentence:
{example["prompt"]} 
"""
message_1 = [
    {"role": "user", "content": user_prompt}, 
    {"role": "assistant", "content": example["completion_1"]},
]
inputs_temp_1 = tokenizer.apply_chat_template(message_1, tokenize=False)
inputs_1 = tokenizer(inputs_temp_1, return_tensors="pt").to(model.device)
detect_token_idx_1 = find_assistant_token_end(inputs_1["input_ids"][0].tolist())

In [5]:
inputs_temp_1

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nComplete the following sentence:\nShe opened the door<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nand saw her friend waiting on the porch.<|eot_id|>'

In [6]:
inputs_1["input_ids"].shape

torch.Size([1, 55])

In [7]:
detect_token_idx_1

44

In [8]:
tokenizer.decode(inputs_1["input_ids"][0][-1])

'<|eot_id|>'

In [9]:
outputs = model(**inputs_1)
logits = outputs.logits

In [10]:
tokenizer.decode(logits[0, -1].argmax().item())

'<|start_header_id|>'

In [11]:
start_token_idx_1 = find_assistant_token_end(inputs_1["input_ids"][0].tolist())
end_token_idx_1 = len(inputs_1["input_ids"][0]) - 2

print("Start token idx:", start_token_idx_1)
print("End token idx:", end_token_idx_1)

Start token idx: 44
End token idx: 53


In [13]:
logits_1 = outputs.logits[0]

In [14]:
log_probs_1 = 1
for token_idx in range(start_token_idx_1, end_token_idx_1): 
    print("Token idx:", token_idx)
    print(f"Detect_Token: {tokenizer.decode(inputs_1['input_ids'][0][token_idx])}")
    print(f"Target_Token: {tokenizer.decode(inputs_1['input_ids'][0][token_idx + 1])}")
    x = torch.log_softmax(logits_1[token_idx], dim=-1)[inputs_1["input_ids"][0][token_idx + 1]].item()
    print(f"Log Prob: {x}")
    
    log_probs_1 = log_probs_1 * x

Token idx: 44
Detect_Token: 


Target_Token: and
Log Prob: -1.8671875
Token idx: 45
Detect_Token: and
Target_Token:  saw
Log Prob: -7.71875
Token idx: 46
Detect_Token:  saw
Target_Token:  her
Log Prob: -2.40625
Token idx: 47
Detect_Token:  her
Target_Token:  friend
Log Prob: -3.890625
Token idx: 48
Detect_Token:  friend
Target_Token:  waiting
Log Prob: -2.109375
Token idx: 49
Detect_Token:  waiting
Target_Token:  on
Log Prob: -0.9609375
Token idx: 50
Detect_Token:  on
Target_Token:  the
Log Prob: -7.915496826171875e-05
Token idx: 51
Detect_Token:  the
Target_Token:  porch
Log Prob: -0.58984375
Token idx: 52
Detect_Token:  porch
Target_Token: .
Log Prob: -3.546875


In [15]:
print("Final Log Prob:", log_probs_1)

Final Log Prob: -0.045290280119493355
